# Job counting for years

In [2]:
!pip install --upgrade py2neo
from py2neo import Graph, Relationship, NodeMatcher
from string import Template
import pandas as pd 

#Login + PW for the local neo4j database
graph = Graph(auth=("neo4j", "sonar2021"))

     |████████████████████████████████| 177 kB 2.5 MB/s eta 0:00:01
  Attempting uninstall: interchange
    Found existing installation: interchange 2021.0.3
    Uninstalling interchange-2021.0.3:
      Successfully uninstalled interchange-2021.0.3
  Attempting uninstall: py2neo
    Found existing installation: py2neo 2021.2.0
    Uninstalling py2neo-2021.2.0:
      Successfully uninstalled py2neo-2021.2.0


In [3]:
startYear = 50 
steps = 50 #how big the time spans will end up in years

# Calculation for the voronois
Birth dates are used for the temporal dimension. The inclusion of death dates would be better though but also slows down the query a lot. Sometimes years without results lead to errors. (A bug that's not solved yet)

## PerName

In [7]:
%%time
#topic term for PerNames (for all: 13min 11s). range defines the number of steps
dfFinal = pd.DataFrame()
for x in range(0, 2): 
    z = x*steps
    startYear = 50
    queryTemplate = Template('MATCH (n:PerName)-[rel:RelationToTopicTerm]-(t:TopicTerm) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN DISTINCT n.Id As ID, t.Name As year_${year}') 
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result).drop_duplicates(subset=None, keep='first', inplace=False)
    df3 = df["year_"+str(startYear+z)].value_counts()
    df4 = pd.DataFrame(df["year_"+str(startYear+z)].value_counts())
    dfFinal = pd.concat([dfFinal, df4], axis=1)

dfFinal.fillna(0)
dfFinal

CPU times: user 18 ms, sys: 5 ms, total: 23 ms
Wall time: 14.1 s


,year_50,year_100
Philosoph,3.0,4.0
Historiker,2.0,NaN
Arzt,2.0,NaN
Rechtsanwalt,2.0,2.0
Schriftsteller,2.0,5.0
Geschichtsschreiber,2.0,NaN
Offizier,1.0,NaN
Chirurg,1.0,NaN
Biograf,1.0,NaN
Politiker,1.0,3.0


In [8]:
dfFinal.index.name = 'EntityName'
dfFinal.to_csv("persontopicterms_50er.csv")

## GeoName

In [9]:
%%time
#Locations for PerName (for all: 14min 48s)
dfFinalGeo = pd.DataFrame()
for x in range(0, 2): 
    z = x*steps
    startYear = 50
    queryTemplate = Template('MATCH (n:PerName)--(t:GeoName) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN DISTINCT n.Id As ID, t.Name As year_${year}') 
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result).drop_duplicates(subset=None, keep='first', inplace=False)
    df3 = df["year_"+str(startYear+z)].value_counts()
    df4 = pd.DataFrame(df["year_"+str(startYear+z)].value_counts())
    dfFinalGeo = pd.concat([dfFinalGeo, df4], axis=1)

dfFinalGeo.fillna(0)
dfFinalGeo

CPU times: user 14.2 ms, sys: 9.39 ms, total: 23.6 ms
Wall time: 21.2 s


,year_50,year_100
Rom,3.0,2.0
Trajanssäule,1.0,NaN
Villa Laurentina,1.0,NaN
Dakien,1.0,NaN
Engelsburg,1.0,NaN
Gallien,1.0,NaN
Wu Liang ci,1.0,NaN
Hildesheim,1.0,NaN
Alexandria,1.0,NaN
Villa di Plinio,1.0,NaN


In [10]:
dfFinalGeo.index.name = 'EntityName'
dfFinalGeo.to_csv("data-results/persongeonames_50er.csv")

## Ressources

In [11]:
%%time
#for all, very slow: > 1h
dfFinalResourceTopic = pd.DataFrame()
for x in range(0, 1): 
    z = x*steps
    startYear = 1000
    queryTemplate = Template('MATCH (n:Resource)--(t:TopicTerm) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN DISTINCT n.Id As ID, t.Name As year_${year}') 
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result).drop_duplicates(subset=None, keep='first', inplace=False)
    df3 = df["year_"+str(startYear+z)].value_counts()
    df4 = pd.DataFrame(df["year_"+str(startYear+z)].value_counts())
    dfFinalResourceTopic = pd.concat([dfFinalResourceTopic, df4], axis=1)

dfFinalResourceTopic.fillna(0)
dfFinalResourceTopic

CPU times: user 135 ms, sys: 145 ms, total: 280 ms
Wall time: 1h 49min 3s


,year_1000,year_1050
Neujahr,1.0,NaN
Zeitschrift,1.0,NaN
Bürgerbeteiligung,1.0,NaN
Fest,1.0,NaN
Bauplanung,1.0,NaN
Hochschulschrift,1.0,NaN
Verkehrsplanung,1.0,NaN
Verkehrsanlage,1.0,NaN
Bibliografie,1.0,NaN
Monografische Reihe,1.0,NaN


In [12]:
dfFinalResourceTopic.index.name = 'EntityName'
dfFinalResourceTopic.to_csv("data-results/resourcetopic_50er.csv")

## MeetName

In [12]:
%%time
#for all, very fast
dfFinalMeetTopic = pd.DataFrame()
for x in range(0, 2): 
    z = x*steps
    startYear = 1000
    queryTemplate = Template('MATCH (n:MeetName)--(t:TopicTerm) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN DISTINCT n.Id As ID, t.Name As year_${year}') 
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result).drop_duplicates(subset=None, keep='first', inplace=False)
    df3 = df["year_"+str(startYear+z)].value_counts()
    df4 = pd.DataFrame(df["year_"+str(startYear+z)].value_counts())
    dfFinalMeetTopic  = pd.concat([dfFinalMeetTopic , df4], axis=1)

dfFinalMeetTopic.fillna(0)
dfFinalMeetTopic

CPU times: user 1.31 s, sys: 40.9 ms, total: 1.35 s
Wall time: 51.3 s


,year_1000,year_1050,year_1100,year_1150,year_1200,year_1250,year_1300,year_1350,year_1400,year_1450,...,year_1550,year_1600,year_1650,year_1700,year_1750,year_1800,year_1850,year_1900,year_1950,year_2000
Synode,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,3.0,1.0,...,4.0,9.0,1.0,1.0,1.0,1.0,8.0,10.0,17.0,23.0
Reichstag,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,4.0,11.0,...,7.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Konzil,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
Hanse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Versammlung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Netzkunst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Internet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Atomphysik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Jugendverband,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [17]:
dfFinalMeetTopic.index.name = 'EntityName'
dfFinalMeetTopic.to_csv("data-results/meetnametopic_50er.csv")

## UniTitle

In [16]:
%%time
#very fast
dfFinalUniTitleTopic = pd.DataFrame()
for x in range(0, 11): 
    z = x*steps
    startYear = 1400
    queryTemplate = Template('MATCH (n:UniTitle)--(t:TopicTerm) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN DISTINCT n.Id As ID, t.Name As year_${year}')
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result).drop_duplicates(subset=None, keep='first', inplace=False)
    df3 = df["year_"+str(startYear+z)].value_counts()
    df4 = pd.DataFrame(df["year_"+str(startYear+z)].value_counts())
    dfFinalUniTitleTopic  = pd.concat([dfFinalUniTitleTopic , df4], axis=1)

dfFinalUniTitleTopic.fillna(0)
dfFinalUniTitleTopic

CPU times: user 146 ms, sys: 4.66 ms, total: 151 ms
Wall time: 11.3 s


,year_1400,year_1450,year_1500,year_1550,year_1600,year_1650,year_1700,year_1750,year_1800,year_1850,year_1900
Stadtrecht,3.0,2.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Friedensvertrag,1.0,1.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,1.0,2.0
Feuerwerksbuch,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Frühneuhochdeutsch,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Halsgerichtsordnung,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Verfassung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Experimentalfilm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Funktechnik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Jugendzeitung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [18]:
dfFinalUniTitleTopic.index.name = 'EntityName'
dfFinalUniTitleTopic.to_csv("data-results/unititletopic_50er.csv")

## CorpName
The first variant is the variant we wanted to use, but couldn't get to work due to connection timeouts, so we had to use the second easier one relying just on the corp names instead of related topics to corp names.

In [42]:
%%time
#Topic Terms for CorpName based on time for connected persons: very very slow. this 
dfFinalCorpTopic = pd.DataFrame()
for x in range(0, 2): 
    z = x*steps
    startYear = 50
    queryTemplate = Template('MATCH (n:PerName)--(c:CorpName)--(t:TopicTerm) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN DISTINCT c.Id As ID, t.Name As year_${year}')
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result).drop_duplicates(subset=None, keep='first', inplace=False)
    df3 = df["year_"+str(startYear+z)].value_counts()
    df4 = pd.DataFrame(df["year_"+str(startYear+z)].value_counts())
    dfFinalCorpTopic  = pd.concat([dfFinalCorpTopic , df4], axis=1)

dfFinalCorpTopic.fillna(0)
dfFinalCorpTopic

CPU times: user 449 ms, sys: 40.7 ms, total: 489 ms
Wall time: 12min 50s


,year_50,year_100,year_150,year_200,year_250,year_300,year_350,year_400,year_450,year_500,...,year_800,year_850,year_900,year_950,year_1000,year_1050,year_1100,year_1150,year_1200,year_1250
Druckerei,54.0,32.0,18.0,10.0,16.0,49.0,22.0,7.0,28.0,6.0,...,7.0,7.0,7.0,8.0,21.0,31.0,24.0,35.0,34.0,43.0
Verlag,50.0,36.0,28.0,15.0,20.0,48.0,30.0,5.0,27.0,7.0,...,10.0,9.0,14.0,12.0,23.0,40.0,31.0,64.0,51.0,65.0
Buchhandel,33.0,15.0,10.0,2.0,6.0,18.0,9.0,2.0,8.0,2.0,...,3.0,2.0,3.0,2.0,4.0,11.0,11.0,13.0,15.0,14.0
Firma,9.0,3.0,2.0,1.0,1.0,3.0,3.0,NaN,2.0,1.0,...,1.0,3.0,2.0,NaN,2.0,4.0,2.0,3.0,3.0,5.0
Sortimentsbuchhandel,7.0,1.0,4.0,1.0,1.0,6.0,3.0,2.0,3.0,NaN,...,3.0,2.0,NaN,NaN,2.0,6.0,4.0,5.0,9.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Frömmigkeit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Griffo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Bibliothèque Nationale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Lazurskij-Kyrillisch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## CorpName version used in visualization

In [ ]:
%%time
#still slow: 17min
dfFinalCorpTopic9 = pd.DataFrame()
for x in range(0, 2): #40
    z = x*steps
    startYear = 50
    queryTemplate = Template('MATCH (n:PerName)--(c:CorpName) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN DISTINCT c.Id As ID, c.Name As year_${year}')
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result).drop_duplicates(subset=None, keep='first', inplace=False)
    df3 = df["year_"+str(startYear+z)].value_counts()
    df4 = pd.DataFrame(df["year_"+str(startYear+z)].value_counts())
    dfFinalCorpTopic9 = pd.concat([dfFinalCorpTopic9, df4], axis=1)

dfFinalCorpTopic2.fillna(0)
dfFinalCorpTopic2

In [47]:
dfFinalCorpTopic2.index.name = 'EntityName'
dfFinalCorpTopic2.to_csv("data-results/corptopic_50er.csv")

## Expemplary counting of number of relations between CorpName to PerName by relation source (here GND)
To get numbers for the other sources (DNB, SBB, KPE) rel.Source needs to be replaced.
the range defines the time spane we look at.

In [7]:
%%time
#PerName -- CorpName by year of perName: GND. Very slow! (hours)
dfFinalCorpPerRel = pd.DataFrame()
for x in range(20, 23): #40
    z = x*steps
    startYear = 50
    queryTemplate = Template('MATCH (n:PerName)-[rel]-(c:CorpName) \
    WHERE (rel.Source = "GND") AND (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN count(rel) As year_${year}') 
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result)
    dfFinalCorpPerRel = pd.concat([dfFinalCorpPerRel, df], axis=1)

dfFinalCorpPerRel

CPU times: user 218 ms, sys: 21.1 ms, total: 239 ms
Wall time: 10min 59s


,year_50,year_100,year_150,year_200,year_250,year_300,year_350,year_400,year_450,year_500,...,year_1050,year_1100,year_1150,year_1200,year_1250,year_1300,year_1350,year_1400,year_1450,year_1500
0,2,0,7,0,2,1,4,0,0,1,...,29,32,33,41,41,37,69,131,300,678


In [25]:
dfFinalCorpPerRel.to_csv("data-results/corp-rel-gnd_50er.csv")

## Example PerName to PerName Relations GND

In [30]:
%%time
#PerName -- PerName by year of perName1: GND
dfFinalPerPerRel1 = pd.DataFrame()
for x in range(0, 10): #40
    z = x*steps
    startYear = 50
    queryTemplate = Template('MATCH (n:PerName)-[rel]-(c:PerName) \
    WHERE (rel.Source = "GND") AND (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN count(rel) As year_${year}') #n.EntityName As PerName, n.EntityId As PerNameID, 
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result)
    dfFinalPerPerRel1 = pd.concat([dfFinalPerPerRel1, df], axis=1)

dfFinalPerPerRel1

CPU times: user 209 ms, sys: 20 ms, total: 229 ms
Wall time: 9min 47s


,year_50,year_100,year_150,year_200,year_250,year_300,year_350,year_400,year_450,year_500,...,year_1050,year_1100,year_1150,year_1200,year_1250,year_1300,year_1350,year_1400,year_1450,year_1500
0,88,207,79,76,66,300,190,68,95,59,...,390,611,665,603,788,857,1294,2911,14524,31592


In [40]:
dfFinalPerPerRel1.to_csv("data-results/per-rel-gnd_50er.csv")

## Example GeoName to PerName Relations GND

In [42]:
%%time
dfFinalGeoPerRel1 = pd.DataFrame()
for x in range(0, 5): 
    z = x*steps
    startYear = 50
    queryTemplate = Template('MATCH (n:PerName)-[rel]-(c:GeoName) \
    WHERE (rel.Source = "GND") AND (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN count(rel) As year_${year}') #n.EntityName As PerName, n.EntityId As PerNameID, 
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result)
    dfFinalGeoPerRel1 = pd.concat([dfFinalGeoPerRel1, df], axis=1)

dfFinalGeoPerRel1

CPU times: user 258 ms, sys: 26.8 ms, total: 285 ms
Wall time: 14min 9s


,year_50,year_100,year_150,year_200,year_250,year_300,year_350,year_400,year_450,year_500,...,year_1300,year_1350,year_1400,year_1450,year_1500,year_1550,year_1600,year_1650,year_1700,year_1750
0,27,12,45,18,15,25,12,13,23,14,...,524,848,1726,4752,10697,22093,30623,40342,44923,58303


In [45]:
dfFinalGeoPerRel1.to_csv("data-results/geo-rel-gnd_50er.csv")

## Example Resource-PerName Relation

In [ ]:
%%time
dfFinalResPerRel1 = pd.DataFrame()
for x in range(0, 1): #40
    z = x*steps
    startYear = 50
    queryTemplate = Template('MATCH (n:Resource)-[rel:RelationToPerName]-(p:PerName) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    AND (rel.Source = "KPE") \
    RETURN count(rel) As year_${year}')
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result)
    dfFinalResPerRel1 = pd.concat([dfFinalResPerRel1, df], axis=1)

dfFinalResPerRel1

## Example for counting computed relations based on their source id

In [20]:
%%time 
##extremly slow
dfFinalResPerRel2 = pd.DataFrame()
for x in range(0, 1): #40
    z = x*steps
    startYear = 50
    queryTemplate = Template('MATCH (n:PerName)-[rel:SocialRelation]-(p:PerName) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    AND ((rel.Source CONTAINS "DE_611") OR (rel.Source CONTAINS "DE-611"))\
    RETURN count(rel) As year_${year}')
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    query
    result = graph.run(query).data()

    df = pd.DataFrame(result)
    dfFinalResPerRel2 = pd.concat([dfFinalResPerRel2, df], axis=1)

dfFinalResPerRel2

CPU times: user 2.95 ms, sys: 9.15 ms, total: 12.1 ms
Wall time: 13.1 s


,year_50
0,16
